In [3]:
# -*- coding: utf-8 -*-

import os, sys

sys.path.append('/home/noel/Documents/unicrawl/')
from crawl.driver import Driver

import time
import re
import json
import progressbar

In [4]:
ugent_driver = Driver()
ugent_driver.init()

LocationParseError: Failed to parse: http://127.0.0.1:48695/session

In [ ]:
# Extract courses (e.g. 'CLAS-B156') and programs (e.g. 'Master') ALREADY crawled -------------------------------
program_fh = open("data/crawling-results/ugent_programs.json")
courses_fh = open("data/crawling-results/ugent_courses.json")
courses = {e["shortname"]: e for e in [json.loads(line) for line in courses_fh.readlines()]}
programs = {e["shortname"]: e for e in [json.loads(line) for line in program_fh.readlines()]}
program_fh.close()
courses_fh.close()
print("already crawled : {} programs, {} courses\n ".format(len(programs.keys()), len(courses.keys())))

program_fh = open("data/crawling-results/ugent_programs.json", "a")
courses_fh = open("data/crawling-results/ugent_courses.json", "a")

# Collecting Bachelor programms list ------------------------------------------------------------------
ugent_driver.driver.get('https://studiegids.ugent.be/2020/EN/FACULTY/faculteiten.html')
time.sleep(5)  # Is it necessary?
# HERE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
programs_ref = [my_elem.get_attribute("href") for my_elem in
                ugent_driver.driver.find_elements_by_xpath("//aside//li[a[@target='_top']]/a")]

for faculte in programs_ref:
    ugent_driver.driver.get(faculte)



for element in programs_ref:
    prg = {}
    prg["name"] = element.find_element_by_tag_name("strong").text
    prg["url"] = element.find_element_by_class_name("item-title__element_title").get_attribute("href")
    prg["faculty"] = element.find_element_by_class_name("item-title__element_title").text
    prg["code"] = element.find_element_by_class_name("search-result__mnemonique").text
    prg["location"] = element.find_element_by_class_name("search-result-formation__separator").text
    programs[prg["code"]] = prg
print("number of bachelors to crawl : %s\n" % len(programs.keys()))
BA_ = len(programs.keys())

# Collecting Master programms list ----------------------------------------------------------------------
ugent_driver.driver.get(
    "https://www.ugent.be/servlet/search?l=0&beanKey=beanKeyRechercheFormation&&types=formation&typeFo=MA&s=FACULTE_ASC&limit=200&page=1")
time.sleep(5)
programs_ref = ugent_driver.driver.find_elements_by_xpath("//div[contains(@class,'search-result__result-item')]")
for element in programs_ref:
    prg = {}
    prg["name"] = element.find_element_by_tag_name("strong").text
    prg["url"] = element.find_element_by_class_name("item-title__element_title").get_attribute("href")
    prg["faculty"] = element.find_element_by_class_name("item-title__element_title").text
    prg["code"] = element.find_element_by_class_name("search-result__mnemonique").text
    prg["location"] = element.find_element_by_class_name("search-result-formation__separator").text
    programs[prg["code"]] = prg

print("number of masters to crawl : %s\n" % (len(programs.keys()) - BA_))
print("number of total programs to crawl : %s\n" % len(programs.keys()))

# Browsing all programs -----------------------------------------------------------------------
for prog_id, prg in programs.items():
    ugent_driver.driver.get(prg["url"] + "#programme")
    prg["courses"] = []
    time.sleep(5)
    print(prg["name"], prg["url"])
    # Listing all courses of the program
    courses_refs = ugent_driver.driver.find_elements_by_xpath("//a[contains(@title,'COURS')]")
    courses_refs = [e.get_attribute("href") for e in courses_refs]
    print(len(courses_refs))
    bar = progressbar.ProgressBar(maxval=len(courses_refs),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    i = 0
    bar.start()
    # Browsing all courses
    for href in courses_refs:
        ugent_driver.driver.get(href)
        time.sleep(5)
        id_ = ugent_driver.driver.find_element_by_class_name("mnemonique").text
        prg["courses"].append(id_)
        # Getting and writing course infos
        if id_ not in courses.keys():
            infos = get_course_infos(href, ugent_driver.driver)
            infos["shortname"] = id_
            courses_fh.write(json.dumps(infos))
            courses_fh.write("\n")
            courses_fh.flush()
            courses[id_] = infos
        i += 1
        bar.update(i)
    if prg["code"] not in programs.keys():
        programs[prg["code"]] = prg
        program_fh.write(json.dumps(prg))
        program_fh.write("\n")
        program_fh.flush()
    bar.finish()
# json.dump(programs,open("data/crawling-results/ugent_programs.json","w"))
# json.dump(courses,open("data/crawling-results/ugent_courses.json","w"))
ugent_driver.delete_driver()
courses_fh.close()
program_fh.close()